In [43]:
import boto3
import pandas as pd
from io import StringIO,BytesIO# bytesio because we are saving in s3 in 
#parquet format whichnwill be usefull in orchastrationa nd airflow
from datetime import datetime,timedelta

In [44]:
arg_date='2022-12-30'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
trg_bucket = 'xetra-sm'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
arg_date_dt=datetime.strptime(arg_date,'%Y-%m-%d').date()-timedelta(days=1)

In [45]:
s3=boto3.resource('s3')
bucket=s3.Bucket('xetra-1234')
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]

In [46]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4980\3669015963.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)


In [47]:
df_all=df_all.loc[:,columns]

In [48]:
df_all.dropna(inplace=True)

# Get opening price per ISIN and day

In [49]:
df_all['opening_price']=df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')

In [50]:
'''df_all.sort_values(by=['Time']): Sorts the DataFrame by Time.
groupby(['ISIN', 'Date']): Groups the DataFrame by ISIN and Date.
transform('first'): Applies the first() function to each group, 
returning the first value in the StartPrice column for each group.'''

"df_all.sort_values(by=['Time']): Sorts the DataFrame by Time.\ngroupby(['ISIN', 'Date']): Groups the DataFrame by ISIN and Date.\ntransform('first'): Applies the first() function to each group, \nreturning the first value in the StartPrice column for each group."

# Get closing price per ISIN and day¶

In [51]:
df_all['closing_price']=df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')

In [52]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2022-12-29,08:00,13.880,13.880,13.880,13.880,63,13.880,14.080
1,DE000A0DJ6J9,2022-12-29,08:00,29.000,29.140,29.000,29.140,6004,29.000,35.540
2,DE000A0D6554,2022-12-29,08:00,13.760,13.880,13.760,13.800,49560,13.760,15.910
3,DE000A0D9PT0,2022-12-29,08:00,212.000,212.000,210.400,210.600,17455,212.000,216.200
4,DE000A0JL9W6,2022-12-29,08:00,66.200,66.250,65.350,66.250,10884,66.200,70.600
...,...,...,...,...,...,...,...,...,...,...
395752,GB00BLD4ZP54,2022-12-31,16:46,19.324,19.324,19.324,19.324,0,18.508,19.324
395753,LU1923627332,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645,10.800,12.400
395754,US98956P1021,2022-12-31,20:30,113.100,113.100,113.100,113.100,0,112.500,113.100
395755,US9224171002,2022-12-31,20:30,24.600,24.600,24.600,24.600,0,24.600,24.600


# Aggregations

In [53]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [54]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-12-29,36.6000,36.700,35.7500,36.700,1773
1,AT000000STR1,2022-12-30,36.6000,36.700,35.7500,36.700,1773
2,AT000000STR1,2022-12-31,36.6000,36.700,35.7500,36.700,1773
3,AT00000FACC2,2022-12-29,8.0500,8.570,7.8700,8.570,10205
4,AT00000FACC2,2022-12-30,8.0500,8.570,7.8700,8.570,10205
...,...,...,...,...,...,...,...
9691,XS2376095068,2022-12-30,34.2880,36.500,34.0640,36.500,1000
9692,XS2376095068,2022-12-31,34.2880,36.500,34.0640,36.500,1000
9693,XS2434891219,2022-12-29,3.4412,3.662,3.4184,3.662,0
9694,XS2434891219,2022-12-30,3.4412,3.662,3.4184,3.662,0


# percent change prev closing

In [55]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [56]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [57]:
df_all = df_all.round(decimals=2)

In [58]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-12-29,36.60,36.70,35.75,36.70,1773,NaN
1,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.0
2,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.0
3,AT00000FACC2,2022-12-29,8.05,8.57,7.87,8.57,10205,NaN
4,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205,0.0
...,...,...,...,...,...,...,...,...
9691,XS2376095068,2022-12-30,34.29,36.50,34.06,36.50,1000,0.0
9692,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.0
9693,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,NaN
9694,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.0


In [59]:
df_all.reset_index(inplace=True)

In [60]:
df_all = df_all[df_all.Date >= arg_date]

In [61]:
df_all

,index,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
1,1,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.0
2,2,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.0
4,4,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205,0.0
5,5,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,0.0
7,7,AT0000606306,2022-12-30,14.51,15.00,13.65,15.28,107836,0.0
...,...,...,...,...,...,...,...,...,...
9689,9689,XS2314660700,2022-12-31,22.26,21.92,21.92,22.28,0,0.0
9691,9691,XS2376095068,2022-12-30,34.29,36.50,34.06,36.50,1000,0.0
9692,9692,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.0
9694,9694,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.0


# save data to s3

In [62]:
key='xetra_smitesh_report' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [63]:
out_buffer=BytesIO()
df_all.to_parquet(out_buffer,index=False)
bucket_target=s3.Bucket('xetra-sm')
bucket_target.put_object(Body=out_buffer.getvalue(),Key=key)

s3.Object(bucket_name='xetra-sm', key='xetra_smitesh_report20240821_212312.parquet')

# read from uploaded file

In [40]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_smitesh_report20240821_210632.parquet


In [64]:
prq_obj = bucket_target.Object(key='xetra_smitesh_report20240821_212312.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [65]:
df_report

,index,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,1,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.0
1,2,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.0
2,4,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205,0.0
3,5,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,0.0
4,7,AT0000606306,2022-12-30,14.51,15.00,13.65,15.28,107836,0.0
...,...,...,...,...,...,...,...,...,...
6459,9689,XS2314660700,2022-12-31,22.26,21.92,21.92,22.28,0,0.0
6460,9691,XS2376095068,2022-12-30,34.29,36.50,34.06,36.50,1000,0.0
6461,9692,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.0
6462,9694,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.0
